In [1]:
!pip install tinygrad numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!python3 -m tinygrad.device

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tinygrad.device' found in sys.modules after import of package 'tinygrad', but prior to execution of 'tinygrad.device'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
  METAL     : FAIL /usr/lib/libobjc.dylib: cannot open shared object file: No such file or directory
  AMD       : FAIL No interface for AMD:0 is available
  NV        : FAIL rm_control returned 58: Invalid structure parameter
* CUDA      : PASS
                 + CUDACompiler (default)
                 + PTXCompiler (CUDA_PTX=1 to make default)
                 + NVCCCompiler (CUDA_NVCC=1 to make default)
  QCOM      : FAIL [Errno 2] No such file or directory: '/dev/kgsl-3d0'
  CL        : FAIL OpenCL Error -1001: Unknown error
  CPU       : PASS
                 + ClangJITCompiler (default)
                 + CPULLVMCompiler (CPU_LLVM=1 to make default)
                 - LVPCompiler: libtinymesa not found (MESA_PATH=/usr/lib). See https://github.

In [3]:
%env DEBUG=2

env: DEBUG=2


In [5]:
from typing import Callable
from tinygrad import Tensor, TinyJit, nn
import numpy as np
import timeit

class Model:
  def __init__(self):
    self.layers: list[Callable[[Tensor], Tensor]] = [
      nn.Conv2d(3, 32, 5), Tensor.relu,
      nn.Conv2d(32, 32, 5), Tensor.relu,
      nn.BatchNorm2d(32), Tensor.max_pool2d,
      nn.Conv2d(32, 64, 3), Tensor.relu,
      nn.Conv2d(64, 64, 3), Tensor.relu,
      nn.BatchNorm2d(64), Tensor.max_pool2d,
      lambda x: x.mean(axis=(2,3)), nn.Linear(64, 1)]

  def __call__(self, x:Tensor) -> Tensor: return x.sequential(self.layers)

if __name__ == "__main__":
  # X = np.random.randn(16, 3, 512, 512).astype(np.float32)
  # Y = np.random.randn(16, 1).astype(np.float32)

  # shortening tensor for speed; doesn't affect kernels
  X, Y = Tensor(np.load("data/X.npy"))[:3], Tensor(np.load("data/Y.npy"))[:3]

  model = Model()
  opt = nn.optim.Adam(nn.state.get_parameters(model))
  
  @TinyJit
  def mae() -> Tensor: return ((model(X).squeeze()-Y)*Y.reciprocal()).abs().mean()*100
  print("starting")
  print(timeit.repeat(mae, repeat=1, number=1))
  print(timeit.repeat(mae, repeat=1, number=1))
  print(timeit.repeat(mae, repeat=1, number=1))
  print(timeit.repeat(mae, repeat=1, number=1))
  print(timeit.repeat(mae, repeat=1, number=1))
  print("done")



starting
scheduled 26 kernels in 55.87 ms
*** CUDA      28 copy   12.58M,    CUDA <- NPY                  arg  2 mem   0.03 GB tm   1018.58us/ 46814.50ms (      0 GFLOPS   12|12     GB/s) 
*** CUDA      29 En7                                            arg  1 mem   0.01 GB tm     18.56us/ 46814.52ms (      0 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      30 copy      128,    CUDA <- NPY                  arg  2 mem   0.01 GB tm     34.78us/ 46814.55ms (      0 GFLOPS    0|0      GB/s) 
*** CUDA      31 E_25_32_3                                      arg  3 mem   0.01 GB tm      7.62us/ 46814.56ms (     83 GFLOPS    1|3      GB/s) ['uniform']
*** CUDA      32 E                                              arg  2 mem   0.01 GB tm      6.50us/ 46814.57ms (      0 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      33 E_8_4                                          arg  3 mem   0.01 GB tm      6.85us/ 46814.57ms (      1 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      34 En1            